In [3]:
!pip install transformers datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00


In [4]:
!pip install bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.5 MB/s eta 0:00:00


In [1]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

In [6]:
!pip install -U bitsandbytes

In [2]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, BitsAndBytesConfig
from PIL import Image
import requests
import torch
import difflib
from datasets import load_dataset
from torch.utils.data import DataLoader
from torchvision import transforms

model_id = "google/paligemma-3b-mix-224"
device = "cuda:0"
dtype = torch.bfloat16

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id, quantization_config=quantization_config
).eval()
processor = AutoProcessor.from_pretrained(model_id)

# Load the A-OKVQA dataset
dataset = load_dataset("HuggingFaceM4/A-OKVQA")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [7]:
def find_most_similar(target, choices):
    # Use SequenceMatcher to calculate similarity
    similarity_scores = [difflib.SequenceMatcher(None, target, choice).ratio() for choice in choices]

    # Get the index of the highest similarity score
    max_index = similarity_scores.index(max(similarity_scores))

    # Return the choice with the highest similarity score
    return choices[max_index]

# Extract the validation split
val_dataset = dataset["validation"]

num_samples = 200
count = 0

for i in range(num_samples):

  image = val_dataset[i]["image"]
  question = val_dataset[i]["question"]
  choices = val_dataset[i]["choices"]
  answer = val_dataset[i]['choices'][val_dataset[i]['correct_choice_idx']]

  model_inputs = processor(text=question, images=image, return_tensors="pt")
  input_len = model_inputs["input_ids"].shape[-1]

  with torch.inference_mode():

      generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
      generation = generation[0][input_len:]
      decoded = processor.decode(generation, skip_special_tokens=True)

  result = find_most_similar(decoded, choices)
  if result == answer:
    count += 1

  print(result, answer)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


cigarette cigarette
one thirty
boiling dirty
icing icing
mobile phone mobile phone
witch devil
drying hair drying hair
crowded crowded
flying crouching
linen denim
united states united states
candle candle
grapes grapes
pills cough drops
american american
books books
afternoon afternoon
eat jump
concert date
fondant fondant
chef chef
frisbee frisbee
dinosaur human
ski resort ski resort
air air
down down
passengers air traffic
japan japan
flatbread pepperoni
walking person closed doors
flea market street festival
white bag white bag
happy birthday happy birthday
baseball baseball
lotions shampoo conditioner
wii boxing wii boxing
bunt bunt
domestic kitchen domestic kitchen
beach savanna
steel steel
dog dog
bathroom bathroom
washing machine sink
meat meat
batting storage
plaid tie dyed
asian asian
decoration call letters
camouflage visibility
to eat to eat
bathroom bathroom
suburban city
probably no absolutely no
bedroom bedroom
family academic
monster truck dirt bikes
new york new york
m

In [8]:
count/num_samples

0.62